# Mapping Project Data Processing

Date: November 22, 2020

Script to automate the cleaning of data collected from Gooogle Search trends

## Initial Setup

In [1]:
# Uncomment this cell to install the dependencies 
# %pip install pandas plotly dash notebook>=5.3 ipywidgets>=7.2

In [1]:
import glob
import os

import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go
import datetime

- Download the dataset from google drive. 
- Create a folder  _input-data_ in the same parent directory as this script and place the expanded _google-trends_ folder there. 
- Create another folder in the same parent directory as this script named _output-data_ 

In [2]:
input_dir = "./input-data/google-trends/"
output_dir = './output-data/'

## Read & Tidy Data

In [3]:
dfs = []

for filepath in glob.glob(input_dir + "*.csv"):
    # read the data 
    filename = filepath.split(os.sep)[-1].strip(".csv")
    country, term, translated_term = filename.split("_")
    df = pd.read_csv(filepath, 
                 index_col=False, 
                 skiprows=3, 
                 infer_datetime_format=True, 
                 names=["date", "score"])
    # add extra columns for tidyness
    df["country"] = country.lower()
    df["term"] = term.lower()
    df["translated_term"] = translated_term.lower()
    dfs.append(df) # add the DataFrame to our list of dfs

In [4]:
# combine all the DataFrames 
data = pd.DataFrame()
for df in dfs:
    data = data.append(df)

In [5]:
# Making dates start at same point in time -- November 12, 2018
data["date"] = data["date"].astype('datetime64[ns]')
start_date = datetime.datetime(2018,11,1)
data = data[data.date >= start_date]

In [6]:
def remove_less_than_one(x):
    if x == "<1":
        return 1
    else:
        return int(x)
data["score"] = data["score"].apply(remove_less_than_one)    

In [7]:
data.iloc[0]["date"].year

2018

In [8]:
def year_change(row):
    """Calculate the change for the same week of the previous year."""
    current_year = row.date.year
    prior_year = current_year - 1 
    week = row.date.isocalendar()[1]
    
    country = row.country
    term = row.term
    current_score = row.score
    
    if row.date > datetime.datetime(2019,10,31): # can't have prior year for the starting_year 
        last_year_row = data[(data.country == country) & (data.term == term) & (pd.DatetimeIndex(data.date).week==week) & (pd.DatetimeIndex(data.date).year == prior_year)]
        last_year_score = last_year_row.score
        difference_score = current_score -  last_year_score 
        return int(difference_score)
    else:
        return None 

In [9]:
data["score_difference"] = data.apply(year_change, axis=1)
data = data[data.score_difference.notna()]

C:\Users\nelea\anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)



In [10]:
print(data.head())
print(data.tail())

         date  score country    term translated_term  score_difference
52 2019-11-03     38     ger  baking          backen               2.0
53 2019-11-10     42     ger  baking          backen               6.0
54 2019-11-17     44     ger  baking          backen               5.0
55 2019-11-24     46     ger  baking          backen               5.0
56 2019-12-01     48     ger  baking          backen               6.0
          date  score country         term translated_term  score_difference
129 2020-10-04      7      uk  toiletpaper     toiletpaper               1.0
130 2020-10-11      6      uk  toiletpaper     toiletpaper              -1.0
131 2020-10-18      7      uk  toiletpaper     toiletpaper               2.0
132 2020-10-25      9      uk  toiletpaper     toiletpaper               3.0
133 2020-11-01      7      uk  toiletpaper     toiletpaper               2.0


In [11]:
data.to_csv(output_dir + "google-trends-difference_NEW.csv", index=False) # save the data

In [12]:
data[(data.country == 'nl') & (data.term == "cooking")].to_csv(output_dir + "nl_check.csv")

## Preliminary Visualization

Super super ugly, but just to emphasize the lack of coding that Plotly requires to get a basic interactive graph up and running: 

In [13]:
# Possibly will be helpful for graphing later 
netherlands = data[data["country"].str.lower() == "nl"]
uk = data[data["country"].str.lower() == "uk"]
germany = data[data["country"].str.lower() == "ger"]

In [14]:
germany_rice_fig = px.line(germany[germany.term.str.lower()=="rice"], x="date", y="score", title="Searches for Rice in Germany")
germany_rice_fig.show()

In [16]:
for country in ["nl", "uk", "ger"]:
    fig = px.line(data[data.country == country], x="date", y="score", facet_row="term", title=country)
    fig.show()